In [13]:
import crawler.register_all_env
import rospy
import gym

import numpy as np
from  collections  import deque
from itertools import count
import torch
import time
#from ddpg_agent import Agent
from ppo_agent import Agent
import matplotlib.pyplot as plt
%matplotlib inline

SEED=0

rospy.init_node('crawler_gyb_ppo', anonymous=True, log_level=rospy.INFO)
torch.manual_seed(SEED)
env = gym.make('CrawlerStandupEnv-v0')
env.reset()
num_inputs = env.observation_space.shape[0]
num_outputs = env.action_space.shape[0]
agent = Agent(state_size=num_inputs, action_size=num_outputs, random_seed=SEED, n_agent=1, \
              fc1_units=128, fc2_units=128)

[INFO] [1611253310.829696, 0.006000]: Start Init ControllersConnection
[INFO] [1611253310.830726, 0.006000]: END Init ControllersConnection


In [14]:
def ppo(env, agent, n_episodes=2000, print_every=10, n_agent = 12):
    """
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        print_every (int): number of episodes to print result
        n_agent (int): number of identical agents in environment
    """
    scores_deque = deque(maxlen=100)
    scores_global = []    
    scores = []
    max_t = 2000
        
    time_start = time.time()
    
    for i_episode in range(1, n_episodes+1):
        states = env.reset()
        agent_scores = np.zeros(n_agent)
        t_max = 0
        for t in range(max_t):
            actions, log_probs, _, values = agent.act(states)
            # get needed information from environment
            next_states, rewards, done = env.step(actions)
            #dones = np.array([1 if t else 0 for t in env_info.local_done])
            agent.save_step([states, values.detach(), actions, log_probs.detach(), rewards, 1 - dones])
            states = next_states
            agent_scores += rewards
            t_max = t
            #if all(dones) or t == max_t-1:
            if done:
                agent.step(next_states)
                break
                
        score = np.mean(agent_scores)
        scores_deque.append(score)
        scores.append(score)
        
        print('Episode: {}, Score: {:.2f}, Max: {:.2f}, Min: {:.2f}, T-max: {}   '\
              .format(i_episode, score, np.max(scores), np.min(scores), t_max))
        
        if i_episode % print_every == 0:
            torch.save(agent.actor_critic.actor.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.actor_critic.critic.state_dict(), 'checkpoint_critic.pth')
            s = (int)(time.time() - time_start) 
            print('*** Episode {} \t Average Score (over agents): {:.2f} \t Average Score on 100 Episode: {:.2f}, Time: {:02}:{:02}:{:02}***'\
                  .format(i_episode, score, np.mean(scores_deque), s//3600, s%3600//60, s%60))
            
        if len(scores_deque) == 100 and np.mean(scores_deque) >= 800: # 650: # 500:   
            break


    print('\n End of PPO!')
    return scores

scores = ppo(env=env, agent=agent, n_agent=1)

ValueError: expected 2D or 3D input (got 1D input)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()